# Schema: XML Schema

Ähnlich wie die Document Type Definition (DTD), ermöglicht auch XML Schema die Spezifikation von XML Dokumente und somit das Validieren solcher Dokumente. Ziel beider Grammatiken ist die Interoperabilität zwischen entwickelten Systeme zu ermöglichen oder erhöhen. In dieser Übung schauen wir uns XML Schema etwas genauer in der Praxis an. Führen Sie zuerst den folgenden Codeblock aus und machen Sie dann der Reihe nach weiter. Beantworten Sie die Fragen (falls zutreffend). Zum Schluss, schreiben Sie ein eigenes XML Schema und ein exemplarisches XML Dokument dafür. Stellen Sie sicher, dass das XML Dokument wohlgeformt und gültig ist.

In [1]:
import io
from lxml import etree as et

def isvalid(xsd, doc):
    print(et.XMLSchema(et.parse(io.StringIO(xsd))).validate(et.fromstring(doc)))

In [2]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="discography"/>
</xs:schema>
"""

isvalid(xsd, '<discography/>')
isvalid(xsd, '<albums/>') # Warum ist dieses Dokument nicht gültig? Antwort: das deklarierte Element "discography" stimmt nicht mit diesem Element "albums" überein.

True
False


In [3]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="title" type="xs:string"/>
</xs:schema>
"""

isvalid(xsd, '<title>The Dark Side of the Moon</title>')
isvalid(xsd, '<title>1973</title>') # Warum ist dieses Dokument gültig? Antwort: Der Elementtyp wird als String definiert. Hier ist allerdings eine Zahl verbaut.

True
True


In [4]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="year" type="xs:decimal"/>
</xs:schema>
"""

isvalid(xsd, '<year>1973</year>')
isvalid(xsd, '<year>The Dark Side of the Moon</year>') # Warum ist dieses Dokument nicht gültig? Antwort: Der Elementtyp wird als decimal angegeben. Hier wird jedoch ein String angegeben. 

True
False


In [5]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="year" type="xs:gYear"/>
</xs:schema>
"""

isvalid(xsd, '<year>1973</year>')

True


In [6]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="year" type="xs:gMonth"/>
</xs:schema>
"""

isvalid(xsd, '<year>1973</year>') # Warum ist dieses Dokument nicht gültig? Antwort: Elementtyp fordert einen Monat, der sich ja bekanntlich aus höchstens zwei Zahlen zusammensetzt. Hier wird allerdings ein vierstelliges Jahr angegben.

False


## Komplexe Elemente Deklarieren: Variante I

In [7]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="title" type="xs:string"/>
        <xs:element name="label" type="xs:string"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc)

True


In [8]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="label" type="xs:string"/>
        <xs:element name="title" type="xs:string"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: Die Reihenfolge der deklarierten Elemente durch xs:sequence stimmt nicht mit dem doc überein. Die Elemente label und titel sind vertauscht.

False


In [9]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="title" type="xs:string"/>
        <xs:element name="label" type="xs:string"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album>
<label>Harvest, EMI</label>
<title>The Dark Side of the Moon</title>
</album>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: Die Reihenfolge der deklarierten Elemente durch xs:sequence stimmt nicht mit dem doc überein. Die Elemente label und titel sind vertauscht.

False


## Komplexe Elemente Deklarieren: Variante II

In [10]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album" type="AlbumType"/>
  
  <xs:complexType name="AlbumType">
    <xs:sequence>
      <xs:element name="title" type="xs:string"/>
      <xs:element name="label" type="xs:string"/>
    </xs:sequence>
  </xs:complexType>

</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc)

True


In [11]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="work" type="WorkType"/>
  <xs:element name="album" type="AlbumType"/>
  <xs:complexType name="WorkType">
    <xs:sequence>
      <xs:element name="title" type="xs:string"/>
    </xs:sequence>
  </xs:complexType>
  <xs:complexType name="AlbumType">
    <xs:complexContent>
      <xs:extension base="WorkType">
        <xs:sequence>
          <xs:element name="label" type="xs:string"/>
        </xs:sequence>
      </xs:extension>
    </xs:complexContent>
 </xs:complexType>
</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc)

doc = """
<album>
<!-- The order matters -->
<label>Harvest, EMI</label>
<title>The Dark Side of the Moon</title>
</album>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: Die Reihenfolge der deklarierten Elemente durch xs:sequence stimmt nicht mit dem doc überein. Die Elemente label und titel sind vertauscht.

True
False


# Attribute Deklarieren

In [12]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:attribute name="title" type="xs:string"/>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album title="The Dark Side of the Moon"/>
"""

isvalid(xsd, doc)

True


In [13]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:simpleContent>
        <xs:extension base="xs:string">
          <xs:attribute name="year" type="xs:gYear" />
        </xs:extension>
      </xs:simpleContent>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album year="1973">The Dark Side of the Moon</album>
"""

isvalid(xsd, doc)

True


# Weitere Eigenschaften

In [14]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="title" type="xs:string" maxOccurs="2"/>
        <xs:element name="label" type="xs:string"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
<title>The Bright Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument gültig? Antwort: durch maxOccurs="2" darf das Element title im Dokument höchstens zweimal auftauchen, was es auch tut.title und label sind in der richtigen Reihenofolge wie es durch xs:sequence festgelegt worden ist.

True


In [15]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:all>
        <xs:element name="title" type="xs:string"/>
        <xs:element name="label" type="xs:string"/>
      </xs:all>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc)

doc = """
<album>
<label>Harvest, EMI</label>
<title>The Dark Side of the Moon</title>
</album>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument gültig? Antwort: durch xs:all wird eine beliebige Reihenfolge der Elemente im Dokument ermöglicht. 

True
True


In [16]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:choice>
        <xs:element name="title" type="xs:string"/>
        <xs:element name="label" type="xs:string"/>
      </xs:choice>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
</album>
"""

isvalid(xsd, doc)

doc = """
<album>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc)

doc = """
<album>
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: durch xs:choice soll nur ein Element von title oder label vorkommen. Da hier beide auftauchen, ist es nicht gültig.

True
True
False


In [17]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="released">
    <xs:simpleType>
      <xs:restriction base="xs:integer">
        <xs:minInclusive value="1950"/>
        <xs:maxInclusive value="2018"/>
      </xs:restriction>
    </xs:simpleType>
  </xs:element>
</xs:schema>
"""

doc = """
<released>1973</released>
"""

isvalid(xsd, doc)

doc = """
<released>2020</released>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: Hier wird eine restriction vorgenommen, wobei ein Wert nicht 2018 überschreiten kann. Da hier der Wert 2020 auftaucht, ist das DOkument ungültig.

True
False


In [18]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="author">
    <xs:simpleType>
      <xs:restriction base="xs:string">
        <xs:enumeration value="Roger Waters"/>
        <xs:enumeration value="David Gilmour"/>
        <xs:enumeration value="Syd Barrett"/>
      </xs:restriction>
    </xs:simpleType>
  </xs:element>
</xs:schema>
"""

doc = """
<author>Roger Waters</author>
"""

isvalid(xsd, doc)

doc = """
<author>Jimmy Page</author>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: Hier wird wieder eine restriction vorgenommen, wobei der value "Jimmy Page" nicht in den deklarierten restrictions auftaucht und somit das Dokument nicht gültig ist.

True
False


In [19]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="author">
    <xs:simpleType>
      <xs:restriction base="xs:string">
        <xs:pattern value="[A-Z][a-z]+"/>
      </xs:restriction>
    </xs:simpleType>
  </xs:element>
</xs:schema>
"""

doc = """
<author>Waters</author>
"""

isvalid(xsd, doc)

doc = """
<author>gilmour</author>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: Das Wertemuster setzt einen Großbuchstaben am Anfang voraus. 

True
False


In [20]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema" 
  targetNamespace="http://example.org"
  elementFormDefault="qualified">
  <xs:element name="album">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="title" type="xs:string"/>
        <xs:element name="label" type="xs:string"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album xmlns="http://example.org">
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc)

True


Denken Sie sich nun ein eigenes XML Dokument aus und erstellen Sie dafür ein XML Schema. Überprüfen Sie das XML Dokument auf Gültigkeit.

In [45]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
<xs:element name="buchsammlung">
<xs:complexType>
<xs:sequence>
<xs:element name="titel" type="xs:string" maxOccurs="2"/>
<xs:element name="autor">
<xs:simpleType>
<xs:restriction base="xs:string">
<xs:pattern value="[A-Z][a-z]+"/>
</xs:restriction>
</xs:simpleType>
</xs:element>
</xs:sequence>
</xs:complexType>
</xs:element>
</xs:schema>
"""

doc = """
<buchsammlung>
<titel>Das leere Haus</titel>
<titel>Das Totenschiff</titel>
<autor>Blackwood</autor>
</buchsammlung>
"""

isvalid(xsd, doc)

True
